In [1]:
import os
from typing import Annotated, Any, Mapping, Optional

import matplotlib.pyplot as plt
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from beartype import beartype as typed
from beartype.door import die_if_unbearable as assert_type
from beartype.typing import Callable, Iterable
from beartype.vale import Is
from datasets import load_dataset
from dvclive.huggingface import DVCLiveCallback
from jaxtyping import Bool, Float, Int
from torch import Tensor as TT
from torch.utils.data import IterableDataset
from transformers import AutoTokenizer

from gpt import DenseGPTConfig, DenseGPTForCausalLM
from utils import explore_batch, fetch_or_ask

%load_ext autoreload
%autoreload 2

In [3]:
gdrive_token = fetch_or_ask("GDRIVE_CREDENTIALS_DATA")
os.environ[
    "DVC_STUDIO_TOKEN"
] = "isat_1mr9HNvqAB6xw8OJ3dXe5O9vMaKol59LCoA5gGP3eLY8NoSF8"

In [5]:
config = DenseGPTConfig(
    vocab_size=502,
    hidden_size=256,
    num_layers=8,
    attention_types=[[["global"], 8]],
    num_heads=16,
    use_dense=True,
)
# model = DenseGPTForCausalLM(config)
model = DenseGPTForCausalLM.from_pretrained("Mlxa/brackets-flat_shuffle", config=config)

Some weights of DenseGPTForCausalLM were not initialized from the model checkpoint at Mlxa/brackets-flat_shuffle and are newly initialized: ['transformer.connections.4', 'transformer.connections.39', 'transformer.connections.14', 'transformer.connections.40', 'transformer.connections.42', 'transformer.connections.28', 'transformer.connections.20', 'transformer.connections.25', 'transformer.connections.13', 'transformer.connections.9', 'transformer.connections.21', 'transformer.connections.27', 'transformer.connections.43', 'transformer.connections.2', 'transformer.connections.36', 'transformer.connections.16', 'transformer.connections.19', 'transformer.connections.7', 'transformer.connections.6', 'transformer.connections.31', 'transformer.connections.10', 'transformer.connections.17', 'transformer.connections.35', 'transformer.connections.18', 'transformer.connections.38', 'transformer.connections.11', 'transformer.connections.34', 'transformer.connections.41', 'transformer.connections

In [2]:
dataset = load_dataset("Mlxa/flat_shuffle")
tokenizer = AutoTokenizer.from_pretrained("Mlxa/brackets-nested")

/home/mike/.local/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [3]:
@typed
def tokenize_function(example: Mapping[str, str | int]) -> Mapping[str, list[int]]:
    result = tokenizer(example["text"])
    result["labels"] = result["input_ids"]
    return result


subset_size = 60000
subset = dataset["train"].select(range(subset_size)).to_iterable_dataset()
tokenized = subset.map(tokenize_function, batched=True).remove_columns(["text"])

In [6]:
explore_batch(model, tokenizer, tokenized)

Mean loss: 2.007
